In [1]:
from libs.connector.postgres_connector import PostgresToJsonConnector
from libs.handler.postgres_handler import PostgresHandler
from libs.handler.json_handler import JsonHandler
from libs.log_handler.log_handler import LogHandler

from libs.workers.algo.exact_address import deduplicate_exact_match_address
from libs.workers.ml.names_addresses.names_addresses import dedupe_io_names_addresses

from libs.dataframes.to_types import to_list_of_dicts


import pandas as pd


log_handler = PostgresHandler(
    env_prefix = 'POSTGRES'
)

logger = LogHandler(db_handler=log_handler)

input_qry = '''
    SELECT
        t3.name,
        t3.id as organization_id,
        t1.id as address_id,
        t1.address_1,
        t1.address_2,
        t2.latitude,
        t2.longitude,
        t1.city,
        t1.state_province,
        t1.postal_code,
        'withinreach' as source
    FROM address t1
    LEFT JOIN location t2
    ON t1.location_id = t2.id
    LEFT JOIN organization t3
    ON t3.id = t2.organization_id
    '''

input_handler = PostgresHandler(
    query = input_qry,
    schema = 'connect211_withinreach_hsds3',
    env_prefix = 'POSTGRES'
)

output_handler = JsonHandler()


In [2]:
conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)

In [3]:
job_id = conn.mutate(
    deduplicate_exact_match_address,
    dedupe_io_names_addresses
)

Importing data ...
Reading from /home/jamey/hackathon/microservice/source/silobuster-dedupe/libs/workers/ml/names_addresses/dedupe_io_addresses_learned_settings
Clustering...


/home/jamey/hackathon/microservice/source/silobuster-dedupe/libs/workers/ml/names_addresses/names_addresses.py:63: FutureWarning: The default value of regex will change from True to False in a future version.
  deduper_df = pandas_dedupe.dedupe_dataframe(df=formatted_df, field_properties=[


# duplicate sets 2098


In [4]:
print (job_id)

beb7f7fe-67e9-4a6d-9bdc-5a13bc6b7d1f
